In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from rapidfuzz.distance import Levenshtein, JaroWinkler
from rapidfuzz.fuzz import token_set_ratio

# Load the CSV file
df = pd.read_csv('bootcamp/string_matching_dataset.csv', header=None, names=['string1', 'string2', 'match'])

# Calculate similarity scores
df['levenshtein_similarity'] = df.apply(lambda row: Levenshtein.normalized_similarity(row['string1'], row['string2']), axis=1)
df['jaro_winkler_similarity'] = df.apply(lambda row: JaroWinkler.normalized_similarity(row['string1'], row['string2']), axis=1)
df['token_set_ratio_similarity'] = df.apply(lambda row: token_set_ratio(row['string1'], row['string2']) / 100, axis=1)

# Prepare the feature matrix and target vector
X = df[['levenshtein_similarity', 'jaro_winkler_similarity', 'token_set_ratio_similarity']]
y = df['match']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the logistic regression model
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred))

# Print the weights for each similarity
weights = model.coef_[0]
print(f'Levenshtein Similarity Weight: {weights[0]}')
print(f'Jaro-Winkler Similarity Weight: {weights[1]}')
print(f'Token Set Ratio Similarity Weight: {weights[2]}')